<a href="https://colab.research.google.com/github/ag497/Heston-Stochastic-Volatility-Model/blob/main/Heston_Stochastic_Volatility_Modelling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [76]:
#IMPORTING NECESSARY LIBRARIES
import yfinance as yf
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
import sympy as sp
import scipy
from scipy import stats
from scipy.stats import norm
from scipy.integrate import quad
from datetime import datetime,timedelta

In [77]:
#GET DATE
def get_date(date_str):
    # date string is in the format 'YYYY-MM-DD'
    date_object = datetime.strptime(date_str, '%Y-%m-%d')

    current_date = datetime.now()
    years_difference = (date_object - current_date).days / 365.25
    return years_difference

In [78]:
#GET TICKER FOR IR
def get_r():
    # 10 year treasury ticker symbol
    treasury_ticker = "^TNX"

    now = datetime.now()
    years_ago = now.replace(year=now.year - 40)

    treasury_data = yf.download(treasury_ticker, start=years_ago, end=now)
    last_yield = treasury_data['Close'].iloc[-1]
    return last_yield

In [79]:
#GET TICKER FOR STOCK
def get_S(symbol):
  stock = yf.Ticker(symbol)
  S_t= stock.history(period="1d")['Close'].iloc[-1]
  return S_t

In [80]:
#LOG RETURNS
def get_R(symbol):
    R = np.log(get_S(symbol))
    return R

In [81]:
#ANNUAL VOLATILITY CALCULATION
def calculate_volatility(symbol):
  data = yf.download(symbol, start=datetime.now().replace(year=datetime.now().year-40), end=datetime.now())
  data['v1']=data['Adj Close'].pct_change()
  data['v2']=data['v1'].pct_change()
  return (data['v1'].std())*np.sqrt(252)

In [82]:
#GET OPTIONS DATA FROM MARKET
def options_data(symbol,date):
    option = yf.Ticker(symbol)
    option = option.option_chain(date)
    return option.calls

In [83]:
# MOM parameter estimation equations
def equations(vars,symbol):
    k, r, sigma, theta = vars
    mu1 = 1 + r
    mu2 = (r + 1)**2 + theta
    mu4 = (1 / (k * (k - 2))) * (
        k**2 * r**4 + 4 * k**2 * r**3 + 6 * k**2 * r**2 * theta - 2 * k * r**4 + 6 * k**2 * r**2 + 12 * k**2 * r * theta
        + 3 * k**2 * theta**2 - 8 * k * r**3 - 12 * k * r * theta + 4 * k**2 * r + 6 * k**2 * theta - 12 * k * r**2
        - 24 * k * r * theta - 6 * k * theta**2 - 3 * sigma**2 * theta + k**2 - 8 * k * r - 12 * k * theta - 2 * k
    )
    mu5 = (1 / (k * (k - 2))) * (
        k**2 * r**5 + 5 * k**2 * r**4 + 10 * k**2 * r**3 * theta - 2 * k * r**5 + 10 * k**2 * r**3 + 30 * k**2 * r**2 * theta
        + 15 * k**2 * r * theta**2 - 10 * k * r**4 - 20 * k * r**3 * theta + 10 * k**2 * r**2 + 30 * k**2 * r * theta + 15 * k**2 * theta**2
        - 20 * k * r**3 - 60 * k * r**2 * theta - 30 * k * r * theta**2 - 15 * sigma**2 * theta + 5 * k**2 * r + 10 * k**2 * theta
        - 20 * k * r**2 - 60 * k * r * theta - 30 * k * theta**2 - 15 * sigma**2 * theta + k**2 - 10 * k * r - 20 * k * theta - 2 * k
    )
    mus=[]
    data = yf.download(symbol, start=datetime.now().replace(year=datetime.now().year-40), end=datetime.now())
    data['change']=data['Adj Close'].pct_change()
    for i in range(5):
      mus.append(np.mean((data['change']+1)**(i+1)))
    return (mu1-mus[0])**2 + (mu2-mus[1])**2+(mu4-mus[3])**2+(mu5-mus[4])**2

In [89]:
# Objective function for optimization
from scipy.optimize import minimize
def optimise(symbol):
  constraints = (
      {'type': 'ineq', 'fun': lambda vars: vars[2]**2},  # sigma^2 > 0
      {'type': 'ineq', 'fun': lambda vars: 2 * vars[0] * vars[3] - vars[2]**2},  # 2k*theta > sigma^2 aka feller condition
      {'type': 'ineq', 'fun': lambda vars: vars[1]}
)

# Initial guess
  initial_guess = [1, 1, 1, 1]

# Solve the optimization problem
  Result=minimize(equations,initial_guess,args=(symbol),method='SLSQP',options={'disp':False}, constraints=constraints)
  #result = minimize(equations, initial_guess,args=(symbol), constraints=constraints)
  if Result.success:
    k_opt, r_opt, sigma_opt, theta_opt = Result.x
    print(f'Optimal values: k = {k_opt:.6f}, r = {r_opt:.6f}, sigma = {sigma_opt:.6f}, theta = {theta_opt:.6f}')
    return Result.x
  else:
    print('Optimization failed.')

In [85]:
i=complex(0,1)

In [86]:
def d(rho, sigma, x):
    a=(rho*sigma*i*x)**2
    b=(sigma**2)*(i*x +x**2)
    return (a+b)**0.5

In [87]:
def g(kappa, rho, sigma, x):
    num=kappa - sigma*rho*i*x +d(rho, sigma, x)
    den=kappa - sigma*rho*i*x -d(rho, sigma, x)
    return num/den

In [88]:
def fHeston(x,r,t,S,rho,kappa, sigma,theta,volvol):
    p1 = np.exp(i*x*r*t)*S**(i*x)
    p2=((1-g(kappa, rho, sigma, x)*np.exp(t*d(rho, sigma, x)))/(1-g(kappa, rho, sigma, x)))**(-2*kappa*theta/(sigma**2))
    p3=(t*kappa*theta/(sigma**2))*(kappa - sigma*rho*i*x -d(rho, sigma,x))
    p4=(volvol/(sigma**2))*(kappa - sigma*rho*i*x -d(rho, sigma, x))*((1-np.exp(-d(rho, sigma, x)*t))/(1-g(kappa, rho, sigma, x)*np.exp(t*-d(rho, sigma, x))))
    return p1*p2*np.exp(p3+p4)

In [38]:
def intHeston(x,r,t,K,kappa,S,rho, sigma, theta,volvol):
    P, iterations, maxnum = 0, 1000, 100
    ds=maxnum/iterations
    for j in range(1,iterations):
        s1 = ds*(2*j+1)/2
        s2 = s1-i
        num1 = np.exp(r*t)*fHeston(s2,r,t,S,rho,kappa, sigma,theta,volvol)
        num2 = K*fHeston(s1,r,t,S,rho,kappa, sigma,theta,volvol)
        den=np.exp(np.log(K) *i*s1)*i*s1
        P += ds*(num1-num2)/den
    P/=np.pi
    P0 = 0.5*(S - K*np.exp(-r*t))
    return np.real(P+P0)

In [95]:
symbol='CRM'
results=optimise(symbol)
option=yf.Ticker(symbol)
date=option.options[1]
calls=options_data(symbol,date)
col_drop=['lastTradeDate', 'volume', 'openInterest', 'contractSize', 'currency']
calls.drop(col_drop, axis=1, inplace=True)
r= r_opt #riskfree IR
x=get_R(symbol) #log of stock price
sigma=calculate_volatility(symbol) #volatility
volvol=sigma_opt #volatility of volatility
S=get_S(symbol) #stock price
t=get_date(date) #time to expiry
kappa = k_opt #mean reverting rate
theta = theta_opt #long term mean volatility
rho = 0.5 #correlation between stock price and volatility
calls['MOM Parameter Heston Pricing']=calls.apply(lambda row:intHeston(x,r,t,row['strike'],kappa,S,rho, sigma, theta,v),axis=1)

[*********************100%%**********************]  1 of 1 completed


In [96]:
calls

,contractSymbol,strike,lastPrice,bid,ask,change,percentChange,impliedVolatility,inTheMoney,MOM Parameter Heston Pricing
0,CRM240705C00160000,160.0,83.98,0.0,0.0,0.0,0.0,0.000010,True,81.200840
1,CRM240705C00170000,170.0,72.00,0.0,0.0,0.0,0.0,0.000010,True,71.379978
2,CRM240705C00190000,190.0,39.50,0.0,0.0,0.0,0.0,0.000010,True,51.342890
3,CRM240705C00195000,195.0,38.21,0.0,0.0,0.0,0.0,0.000010,True,46.017038
4,CRM240705C00200000,200.0,33.00,0.0,0.0,0.0,0.0,0.000010,True,40.561967
5,CRM240705C00205000,205.0,40.00,0.0,0.0,0.0,0.0,0.000010,True,35.104288
6,CRM240705C00210000,210.0,33.10,0.0,0.0,0.0,0.0,0.000010,True,29.825778
7,CRM240705C00215000,215.0,27.38,0.0,0.0,0.0,0.0,0.000010,True,24.913818
8,CRM240705C00220000,220.0,20.75,0.0,0.0,0.0,0.0,0.000010,True,20.516680
9,CRM240705C00222500,222.5,8.70,0.0,0.0,0.0,0.0,0.000010,True,18.540333


In [104]:
#MLE PARAMETER ESTIMATION
import numpy as np
from scipy.optimize import minimize
from datetime import datetime
import yfinance as yf

# Function to fetch data and calculate log returns and volatility
def fetch_data_and_calculate(symbol):
    # Fetch historical data
    data = yf.download(symbol, start=datetime.now().replace(year=datetime.now().year-20), end=datetime.now())

    # Calculate log returns for Adj Close
    data['log_return'] = np.log(data['Adj Close']) - np.log(data['Adj Close'].shift(1))
    # Drop NaN values
    data.dropna(inplace=True)

    window = 252  # Adjust this window size as needed
    data['volatility'] = data['log_return'].rolling(window=window).std() * np.sqrt(252)
    # Drop NaN values
    data.dropna(inplace=True)

    return data['log_return'].values, data['volatility'].values

# Likelihood function
def log_likelihood(vars, symbol):
    r, k, theta, sigma, rho = vars
    log_returns, volatility = fetch_data_and_calculate(symbol)
    n = len(log_returns)
    log_likelihood_value = 0

    epsilon = 1e-8  # Small epsilon to prevent division by zero

    for t in range(n):
        if t == 0:
            Qt_prev = 0  # Assuming initial value for Qt is 0 (log return starts from 0)
            Vt_prev = np.var(log_returns)  # Initial guess for Vt based on log returns
        else:
            Qt_prev = log_returns[t - 1]
            Vt_prev = volatility[t - 1]

        Qt_curr = log_returns[t]
        Vt_curr = volatility[t]

        # Joint probability density function
        if sigma > 0 and Vt_prev > 0 and 1 - rho**2 > epsilon:
            part1 = -0.5 * np.log(2 * np.pi) - np.log(sigma) - 0.5 * np.log(Vt_prev) - 0.5 * np.log(1 - rho**2)
            part2 = -0.5 * ((Qt_curr - (1 + r))**2) / (Vt_curr * (1 - rho**2))
            part3 = (rho * (Qt_curr - (1 + r)) * (Vt_curr - Vt_prev - k * (theta - Vt_prev))) / (Vt_curr * sigma * (1 - rho**2))
            part4 = -0.5 * ((Vt_curr - Vt_prev - k * (theta - Vt_prev))**2) / (sigma**2 * Vt_curr * (1 - rho**2))

            log_likelihood_value += part1 + part2 + part3 + part4
        else:
            # Handle cases where logarithm or division operations are undefined
            # For example, you can skip the contribution to log_likelihood_value
            # or set part1 to a large negative value to penalize such cases
            log_likelihood_value += -np.inf  # Large negative value or any suitable action

    return -log_likelihood_value  # Minimize negative log-likelihood

# Initial guess for parameters
initial_guess = [1, 1, 1, 1, 0.8]  # Initial guesses for r, k, theta, sigma, rho

# Constraints
constraints = (
    {'type': 'ineq', 'fun': lambda vars: vars[3]**2},  # sigma^2 > 0
    {'type': 'ineq', 'fun': lambda vars: 2 * vars[1] * vars[2] - vars[3]**2},  # 2k*theta > sigma^2 (Feller condition)
    {'type': 'ineq', 'fun': lambda vars: vars[0]}  # r > 0
)

# Solve the optimization problem
result = minimize(log_likelihood, initial_guess,args=(symbol,), constraints=constraints)

if result.success:
    r_opt, k_opt, theta_opt, sigma_opt, rho_opt = result.x
    print(f'Optimal values: r = {r_opt:.6f}, k = {k_opt:.6f}, theta = {theta_opt:.6f}, sigma = {sigma_opt:.6f}, rho = {rho_opt:.6f}')
else:
    print('Optimization failed.')


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Optimal values: r = -0.000000, k = 0.003127, theta = 0.567658, sigma = 0.012784, rho = 0.048889


In [106]:
r=r_opt #risk free IR
x=get_R(symbol) #log of stock price
sigma=calculate_volatility(symbol) #volatility
volvol=sigma_opt #volatility of volatility
S=get_S(symbol) #stock price
t=get_date(date) #time to expiry
kappa = k_opt #mean reverting rate
theta = theta_opt #long term mean volatility
rho = rho_opt #correlation between stock price and volatility
calls['MLE Parameter Heston Pricing']=calls.apply(lambda row:intHeston(x,r,t,row['strike'],kappa,S,rho, sigma, theta,v),axis=1)

[*********************100%%**********************]  1 of 1 completed


In [107]:
calls

,contractSymbol,strike,lastPrice,bid,ask,change,percentChange,impliedVolatility,inTheMoney,MOM Parameter Heston Pricing,MLE Parameter Heston Pricing
0,CRM240705C00160000,160.0,83.98,0.0,0.0,0.0,0.0,0.000010,True,81.200840,81.942674
1,CRM240705C00170000,170.0,72.00,0.0,0.0,0.0,0.0,0.000010,True,71.379978,72.204747
2,CRM240705C00190000,190.0,39.50,0.0,0.0,0.0,0.0,0.000010,True,51.342890,52.588565
3,CRM240705C00195000,195.0,38.21,0.0,0.0,0.0,0.0,0.000010,True,46.017038,47.674120
4,CRM240705C00200000,200.0,33.00,0.0,0.0,0.0,0.0,0.000010,True,40.561967,42.779898
5,CRM240705C00205000,205.0,40.00,0.0,0.0,0.0,0.0,0.000010,True,35.104288,37.935490
6,CRM240705C00210000,210.0,33.10,0.0,0.0,0.0,0.0,0.000010,True,29.825778,33.185067
7,CRM240705C00215000,215.0,27.38,0.0,0.0,0.0,0.0,0.000010,True,24.913818,28.587356
8,CRM240705C00220000,220.0,20.75,0.0,0.0,0.0,0.0,0.000010,True,20.516680,24.212052
9,CRM240705C00222500,222.5,8.70,0.0,0.0,0.0,0.0,0.000010,True,18.540333,22.130887


In [116]:
import numpy as np
from scipy.stats import norm

def black_scholes_call_price(S_t, K, tau, r, v_t):
    """
    Calculate the Black-Scholes European call option price.

    Parameters:
    S_t (float): Current stock price
    K (float): Strike price
    tau (float): Time to maturity (in years)
    r (float): Risk-free interest rate
    sigma (float): Volatility of the underlying asset

    Returns:
    float: Call option price
    """
    d1 = (np.log(S_t / K) + (r + 0.5 * v_t*2) * tau) / (v_t*np.sqrt(tau))
    d2 = d1 - v_t * np.sqrt(tau)

    call_price = (S_t * norm.cdf(d1) - K * np.exp(-r * tau) * norm.cdf(d2))
    return call_price


#call_price_bs = black_scholes_call_price(S_t, K, tau, r, v_t)
#print(f'Black-Scholes Call Price: {call_price_bs:.6f}')

In [117]:
r=get_r() #risk free IR
v_t=calculate_volatility(symbol) #volatility
S_t=get_S(symbol) #stock price
tau=get_date(date) #time to expiry
calls['BSM Pricing']=calls.apply(lambda row:black_scholes_call_price(S_t, row['strike'], tau, r, v_t),axis=1)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [118]:
calls

,contractSymbol,strike,lastPrice,bid,ask,change,percentChange,impliedVolatility,inTheMoney,MOM Parameter Heston Pricing,MLE Parameter Heston Pricing,MAE of MOM,MAE of MLE,BSM Pricing
0,CRM240705C00160000,160.0,83.98,0.0,0.0,0.0,0.0,0.000010,True,81.200840,81.942674,2.779160,2.037326,9.585198e+01
1,CRM240705C00170000,170.0,72.00,0.0,0.0,0.0,0.0,0.000010,True,71.379978,72.204747,0.620022,0.204747,8.664585e+01
2,CRM240705C00190000,190.0,39.50,0.0,0.0,0.0,0.0,0.000010,True,51.342890,52.588565,11.842890,13.088565,6.823360e+01
3,CRM240705C00195000,195.0,38.21,0.0,0.0,0.0,0.0,0.000010,True,46.017038,47.674120,7.807038,9.464120,6.363053e+01
4,CRM240705C00200000,200.0,33.00,0.0,0.0,0.0,0.0,0.000010,True,40.561967,42.779898,7.561967,9.779898,5.902747e+01
5,CRM240705C00205000,205.0,40.00,0.0,0.0,0.0,0.0,0.000010,True,35.104288,37.935490,4.895712,2.064510,5.442443e+01
6,CRM240705C00210000,210.0,33.10,0.0,0.0,0.0,0.0,0.000010,True,29.825778,33.185067,3.274222,0.085067,4.982146e+01
7,CRM240705C00215000,215.0,27.38,0.0,0.0,0.0,0.0,0.000010,True,24.913818,28.587356,2.466182,1.207356,4.521892e+01
8,CRM240705C00220000,220.0,20.75,0.0,0.0,0.0,0.0,0.000010,True,20.516680,24.212052,0.233320,3.462052,4.061814e+01
9,CRM240705C00222500,222.5,8.70,0.0,0.0,0.0,0.0,0.000010,True,18.540333,22.130887,9.840333,13.430887,3.831958e+01


In [123]:
# Calculate MAE between last Price and MOM Parameter Heston Pricing
calls['MAE of MOM'] = np.abs(calls['lastPrice'] - calls['MOM Parameter Heston Pricing'])

# Calculate MAE between last Price and MLE Parameter Heston Pricing
calls['MAE of MLE'] = np.abs(calls['lastPrice'] - calls['MLE Parameter Heston Pricing'])

#Calculate MAE between last Price and BSM Pricing
calls['MAE of BSM'] = np.abs(calls['lastPrice'] - calls['BSM Pricing'])

# Calculate MSE
calls['MSE of MOM'] = np.square(calls['lastPrice'] - calls['MOM Parameter Heston Pricing'])
calls['MSE of MLE'] = np.square(calls['lastPrice'] - calls['MLE Parameter Heston Pricing'])
calls['MSE of BSM'] = np.square(calls['lastPrice'] - calls['BSM Pricing'])

# Calculate RMSE
calls['RMSE of MOM'] = np.sqrt(calls['MSE of MOM'])
calls['RMSE of MLE'] = np.sqrt(calls['MSE of MLE'])
calls['RMSE of BSM'] = np.sqrt(calls['MSE of BSM'])

In [124]:
calls

,contractSymbol,strike,lastPrice,bid,ask,change,percentChange,impliedVolatility,inTheMoney,MOM Parameter Heston Pricing,...,MAE of MOM,MAE of MLE,BSM Pricing,MAE of BSM,MSE of MOM,MSE of MLE,MSE of BSM,RMSE of MOM,RMSE of MLE,RMSE of BSM
0,CRM240705C00160000,160.0,83.98,0.0,0.0,0.0,0.0,0.000010,True,81.200840,...,2.779160,2.037326,9.585198e+01,11.871976,7.723731,4.150697,140.943813,2.779160,2.037326,11.871976
1,CRM240705C00170000,170.0,72.00,0.0,0.0,0.0,0.0,0.000010,True,71.379978,...,0.620022,0.204747,8.664585e+01,14.645850,0.384428,0.041921,214.500917,0.620022,0.204747,14.645850
2,CRM240705C00190000,190.0,39.50,0.0,0.0,0.0,0.0,0.000010,True,51.342890,...,11.842890,13.088565,6.823360e+01,28.733598,140.254040,171.310544,825.619630,11.842890,13.088565,28.733598
3,CRM240705C00195000,195.0,38.21,0.0,0.0,0.0,0.0,0.000010,True,46.017038,...,7.807038,9.464120,6.363053e+01,25.420535,60.949836,89.569562,646.203585,7.807038,9.464120,25.420535
4,CRM240705C00200000,200.0,33.00,0.0,0.0,0.0,0.0,0.000010,True,40.561967,...,7.561967,9.779898,5.902747e+01,26.027474,57.183340,95.646397,677.429377,7.561967,9.779898,26.027474
5,CRM240705C00205000,205.0,40.00,0.0,0.0,0.0,0.0,0.000010,True,35.104288,...,4.895712,2.064510,5.442443e+01,14.424425,23.967998,4.262203,208.064050,4.895712,2.064510,14.424425
6,CRM240705C00210000,210.0,33.10,0.0,0.0,0.0,0.0,0.000010,True,29.825778,...,3.274222,0.085067,4.982146e+01,16.721460,10.720532,0.007236,279.607229,3.274222,0.085067,16.721460
7,CRM240705C00215000,215.0,27.38,0.0,0.0,0.0,0.0,0.000010,True,24.913818,...,2.466182,1.207356,4.521892e+01,17.838917,6.082055,1.457708,318.226946,2.466182,1.207356,17.838917
8,CRM240705C00220000,220.0,20.75,0.0,0.0,0.0,0.0,0.000010,True,20.516680,...,0.233320,3.462052,4.061814e+01,19.868142,0.054438,11.985801,394.743086,0.233320,3.462052,19.868142
9,CRM240705C00222500,222.5,8.70,0.0,0.0,0.0,0.0,0.000010,True,18.540333,...,9.840333,13.430887,3.831958e+01,29.619575,96.832145,180.388736,877.319229,9.840333,13.430887,29.619575


In [125]:
mae_mean = calls[['MAE of MOM', 'MAE of MLE', 'MAE of BSM']].mean()
mse_mean = calls[['MSE of MOM', 'MSE of MLE', 'MSE of BSM']].mean()
rmse_mean = calls[['RMSE of MOM', 'RMSE of MLE', 'RMSE of BSM']].mean()

print("Mean Absolute Error (MAE):\n", mae_mean)
print("\nMean Squared Error (MSE):\n", mse_mean)
print("\nRoot Mean Squared Error (RMSE):\n", rmse_mean)

Mean Absolute Error (MAE):
 MAE of MOM    2.816887
MAE of MLE    3.053207
MAE of BSM    9.707236
dtype: float64

Mean Squared Error (MSE):
 MSE of MOM     14.269009
MSE of MLE     19.156207
MSE of BSM    180.733994
dtype: float64

Root Mean Squared Error (RMSE):
 RMSE of MOM    2.816887
RMSE of MLE    3.053207
RMSE of BSM    9.707236
dtype: float64


In [129]:
#R SQUARED PARAMETER

## Calculate SST
y_mean = calls['lastPrice'].mean()
calls['SST'] = np.square(calls['lastPrice'] - y_mean)
SST = calls['SST'].sum()

# Calculate SSR for each model
calls['SSR of MOM'] = np.square(calls['lastPrice'] - calls['MOM Parameter Heston Pricing'])
calls['SSR of MLE'] = np.square(calls['lastPrice'] - calls['MLE Parameter Heston Pricing'])
calls['SSR of BSM'] = np.square(calls['lastPrice'] - calls['BSM Pricing'])

SSR_MOM = calls['SSR of MOM'].sum()
SSR_MLE = calls['SSR of MLE'].sum()
SSR_BSM = calls['SSR of BSM'].sum()

# Calculate R^2 for each model
R2_MOM = 1 - (SSR_MOM / SST)
R2_MLE = 1 - (SSR_MLE / SST)
R2_BSM = 1 - (SSR_BSM / SST)

print("R^2 for MOM Parameter Heston Pricing:", R2_MOM)
print("R^2 for MLE Parameter Heston Pricing:", R2_MLE)
print("R^2 for BSM Pricing:", R2_BSM)


R^2 for MOM Parameter Heston Pricing: 0.9593502546443952
R^2 for MLE Parameter Heston Pricing: 0.9454275381754474
R^2 for BSM Pricing: 0.48512256528775954


In [142]:
import numpy as np
import pandas as pd

# Filter for out of the money calls
OTM_calls = calls[~calls['inTheMoney']].copy()

# Calculate absolute errors
OTM_calls.loc[:, 'MAE of MOM'] = np.abs(OTM_calls['lastPrice'] - OTM_calls['MOM Parameter Heston Pricing'])
OTM_calls.loc[:, 'MAE of MLE'] = np.abs(OTM_calls['lastPrice'] - OTM_calls['MLE Parameter Heston Pricing'])
OTM_calls.loc[:, 'MAE of BSM'] = np.abs(OTM_calls['lastPrice'] - OTM_calls['BSM Pricing'])

# Calculate improvement
improvement_MOM = OTM_calls['MAE of BSM'].mean() - OTM_calls['MAE of MOM'].mean()
improvement_MLE = OTM_calls['MAE of BSM'].mean() - OTM_calls['MAE of MLE'].mean()
improvement_heston = OTM_calls['MAE of MOM'].mean() - OTM_calls['MAE of MLE'].mean()

#display
print("Average Improvement of MOM over BSM for out-of-the-money options:", improvement_MOM)
print("Average Improvement of MLE over BSM for out-of-the-money options:", improvement_MLE)
print("Average Improvement of MLE over MOM for out-of-the-money options:", improvement_heston)

Average Improvement of MOM over BSM for out-of-the-money options: 1.2675772733076895
Average Improvement of MLE over BSM for out-of-the-money options: 1.5889965221542406
Average Improvement of MLE over MOM for out-of-the-money options: 0.32141924884655104


In [141]:
import numpy as np
import pandas as pd

# Filter for in the money calls
ITM_calls = calls[calls['inTheMoney']].copy()

# Calculate absolute errors
ITM_calls.loc[:, 'MAE of MOM'] = np.abs(ITM_calls['lastPrice'] - ITM_calls['MOM Parameter Heston Pricing'])
ITM_calls.loc[:, 'MAE of MLE'] = np.abs(ITM_calls['lastPrice'] - ITM_calls['MLE Parameter Heston Pricing'])
ITM_calls.loc[:, 'MAE of BSM'] = np.abs(ITM_calls['lastPrice'] - ITM_calls['BSM Pricing'])

# Calculate improvement of MOM over BSM
improvement_MOM_ITM = ITM_calls['MAE of BSM'].mean() - ITM_calls['MAE of MOM'].mean()
improvement_MLE_ITM = ITM_calls['MAE of BSM'].mean() - ITM_calls['MAE of MLE'].mean()
improvement_heston_ITM = ITM_calls['MAE of MLE'].mean() - ITM_calls['MAE of MOM'].mean()

#display
print("Average Improvement of MOM over BSM for in-the-money options:", improvement_MOM_ITM)
print("Average Improvement of MLE over BSM for in-the-money options:", improvement_MLE_ITM)
print("Average Improvement of MLE over MOM for in-the-money options:", improvement_heston_ITM)

Average Improvement of MOM over BSM for in-the-money options: 15.636880377192249
Average Improvement of MLE over BSM for in-the-money options: 14.532967618098976
Average Improvement of MLE over MOM for in-the-money options: 1.103912759093273


CONCLUSION

---



We observe that Heston model outperforms Black Scholes model in many combinations of moneyness and time-to-maturity combinations.

As time to maturity increases, Heston Model starts outperforming Black Scholes Model

Heston Model provides better estimates in case of ITM options as time-to-maturity increases.

For OTM options, Heston model gives significantly better estimates aka way closer to market prices than Black Scholes model. In fact, BSM performs very poorly in case of OTM options, given the high MAE values.

The code can be improved further by focusing on better calibration and parameter estimation techniques to further improve pricing.

The next projects undertaken or next step undertaken in study of heston volatility modelling would be to perform sensitivity analysis and also use the parameters obtained to try and price exotic options.


References:

https://www.valpo.edu/mathematics-statistics/files/2015/07/Estimating-Option-Prices-with-Heston%E2%80%99s-Stochastic-Volatility-Model.pdf

https://uregina.ca/~kozdron/Teaching/Regina/441Fall14/Notes/L35-Nov28.pdf

https://calebmigosi.medium.com/